In [29]:
import os
import re
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [17]:
pd.options.plotting.backend = "plotly"
template = "plotly_dark"

In [18]:
def load_price_timeseries(file: str) -> pd.Series:
    df = pd.read_csv(file)
    df.index = pd.to_datetime(df["Date"], format="%d/%m/%Y %H:%M")
    return df["Intraday Continuous 15 minutes ID1-Price"]  # * 1e-6 # €/MWh -> €/Wh

In [19]:
def load_imbalance_prices(file: str) -> pd.Series:
    df = pd.read_csv(file, sep=";", decimal=",")
    t = pd.to_datetime(df["Datum"] + " " + df["von"], format='%d.%m.%Y %H:%M') 
    df["datetime"] = pd.date_range(t.iloc[0], t.iloc[-1], freq="15Min") # avoid repeated indices from daylight saving time change
    df = df.set_index("datetime")
    return df

In [20]:
def load_results(dir):
    res = {}
    files = [f for f in os.listdir(dir) if f.endswith(".parquet")]
    for file in files:
        name, _ = os.path.splitext(file)
        res[name] = pd.read_parquet(dir + file)

    return res

In [23]:
def exctract_value(string, key):
    match = re.search(fr"{key}=([\d.]+)", string)
    if match:
        value = match.group(1)
    return float(value)

In [21]:
def calculate_revenue(df, price):
    price = price.resample("1Min").ffill()
    df = df.join(price)
    return -1 * sum(df["power_opt"] * df["Intraday Continuous 15 minutes ID1-Price"]) * (1/60) * 1e-6  # W -> MWh

In [8]:
def calculate_imbalance_cost(df, price):
    price = price.resample("1Min").ffill()
    df = df.join(price)
    df["imb"] = -(df["power_opt"] - df["power_sim"]) * (1 / 60) * 1e-6 # MWh
    # negation since positive power is charging
    
    # BESS under-supply
    df_u = df.loc[df.imb > 0]
    cost_u = sum(df_u["imb"] * df_u["reBAP unterdeckt"])
    
    # BESS over-supply
    df_o = df.loc[df.imb < 0]
    cost_o = sum(df_o["imb"] * df_o["reBAP ueberdeckt"])

    return cost_u + cost_o

In [9]:
def analyze_imbalance(df):
    df = df.copy()
    df["imb"] = -(df["power_opt"] - df["power_sim"])
    imb_under = df.loc[df.imb > 0, "imb"].sum() * (1/60)
    imb_over  = df.loc[df.imb < 0, "imb"].sum() * (1/60)
    print(f"{imb_under=:.2f}")
    print(f"{imb_over=:.2f}")

In [31]:
def plot_timeseries(df, **kwargs):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    fig.update_layout(**kwargs)

    for id in ["soc_sim", "soc_opt"]:
        fig.add_trace(go.Scatter(x=df.index, y=df[id], name=id), row=1, col=1)

    for id in ["power_sim", "power_opt"]:
        fig.add_trace(go.Scatter(x=df.index, y=df[id], name=id), row=2, col=1)
    
    return fig

In [53]:
def plot_imbalance(df, **kwargs):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
    fig.update_layout(**kwargs)

    for id in ["power_sim", "power_opt"]:
        fig.add_trace(go.Scatter(x=df.index, y=df[id], name=id), row=1, col=1)    

    for id in ["soc_sim", "soc_opt"]:
        fig.add_trace(go.Scatter(x=df.index, y=df[id], name=id), row=2, col=1)

    imb = (df["power_opt"] - df["power_sim"])
    fig.add_trace(go.Scatter(x=df.index, y=imb, name="imb"), row=3, col=1)

    return fig

In [67]:
def plot_diff(df1, df2, **kwargs):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    fig.update_layout(**kwargs)

    for (i, df) in enumerate([df1, df2]):
        fig.add_trace(go.Scatter(x=df.index, y=df["power_opt"], name=f"power {i}"), row=1, col=1)    

    for (i, df) in enumerate([df1, df2]):
        fig.add_trace(go.Scatter(x=df.index, y=df["soc_opt"], name=f"soc {i}"), row=2, col=1)

    return fig

In [10]:
price = load_price_timeseries("data/intraday_prices/electricity_prices_germany_2021.csv")

In [11]:
imb = load_imbalance_prices("data/balancing_energy_prices/reBAP_2021.csv")

In [25]:
dir = "results/diff/"
res = load_results(dir)

In [60]:
dt = 900
# plot_timeseries(res[f"2021 NL fec=1.5 r=1.0 r_opt=1.0 {dt=}"], template=template)
# plot_imbalance(res[f"2021 NL fec=1.5 r=1.0 r_opt=1.0 {dt=}"], template=template, height=800)

In [61]:
dt = 60
# plot_timeseries(res[f"2021 NL fec=1.5 r=1.0 r_opt=1.0 {dt=}"], template=template)
# plot_imbalance(res[f"2021 NL fec=1.5 r=1.0 r_opt=1.0 {dt=} - nl"], template=template, height=800)

In [68]:
plot_diff(
    res[f"2021 NL fec=1.5 r=1.0 r_opt=1.0 dt=900"],
    res[f"2021 NL fec=1.5 r=1.0 r_opt=1.0 dt=900 - nl"],
)

In [26]:
for dt in (900, 300, 180, 60):
    df = res[f"2021 NL fec=1.5 r=1.0 r_opt=1.0 {dt=}"]
    print(f"{dt = }")
    print("Charge")
    df_ch = df[df["power_opt"] > 0]
    analyze_imbalance(df_ch)
    print("Dicsharge")
    df_dch = df[df["power_opt"] < 0]
    analyze_imbalance(df_dch)
    print("")

dt = 900
Charge
imb_under=0.00
imb_over=-112.20
Dicsharge
imb_under=173.17
imb_over=0.00

dt = 300
Charge
imb_under=0.00
imb_over=-209.75
Dicsharge
imb_under=543.06
imb_over=0.00

dt = 180
Charge
imb_under=0.00
imb_over=-210.64
Dicsharge
imb_under=909.79
imb_over=0.00

dt = 60
Charge
imb_under=0.00
imb_over=-200.82
Dicsharge
imb_under=985.33
imb_over=0.00



In [27]:
for dt in (900, 300, 180, 60):
    df = res[f"2021 NL fec=1.5 r=1.0 r_opt=1.0 {dt=} - nl"]
    print(f"{dt = }")
    print("Charge")
    df_ch = df[df["power_opt"] > 0]
    analyze_imbalance(df_ch)
    print("Dicsharge")
    df_dch = df[df["power_opt"] < 0]
    analyze_imbalance(df_dch)
    print("")

dt = 900
Charge
imb_under=0.00
imb_over=-705.91
Dicsharge
imb_under=0.00
imb_over=0.00

dt = 300
Charge
imb_under=0.00
imb_over=-338.06
Dicsharge
imb_under=0.00
imb_over=0.00

dt = 180
Charge
imb_under=0.00
imb_over=-928.55
Dicsharge
imb_under=21.11
imb_over=0.00

dt = 60
Charge
imb_under=0.00
imb_over=-813.70
Dicsharge
imb_under=0.00
imb_over=0.00



In [15]:
for (id, df) in res.items():
    print(id)
    rev = calculate_revenue(df, price)
    print(f"{rev=:.2f}")
    bal = calculate_imbalance_cost(df, imb)
    print(f"{bal=:.2f}")
    total = rev - bal
    print(f"{total=:.2f}")
    print("")

2021 NL fec=1.5 r=1.0 r_opt=1.0 dt=180 - nl
rev=100.29
bal=-0.02
total=100.31

2021 NL fec=1.5 r=1.0 r_opt=1.0 dt=180
rev=85.04
bal=0.02
total=85.03

2021 NL fec=1.5 r=1.0 r_opt=1.0 dt=300 - nl
rev=99.36
bal=-0.01
total=99.37

2021 NL fec=1.5 r=1.0 r_opt=1.0 dt=300
rev=84.75
bal=0.01
total=84.74

2021 NL fec=1.5 r=1.0 r_opt=1.0 dt=60
rev=85.04
bal=0.02
total=85.01

2021 NL fec=1.5 r=1.0 r_opt=1.0 dt=900 - nl
rev=97.56
bal=-0.05
total=97.61

2021 NL fec=1.5 r=1.0 r_opt=1.0 dt=900
rev=84.70
bal=0.01
total=84.70

